In [ ]:
# 1. Star
# 2. Folk
# 3. Watching
# 4. Commits
# 5. Issue
# 6. PL 
# 7. Pull Request
# 8. Actions 
# 9. Docs 
# 10. Contributor
# 11. Code

In [42]:
import pandas as pd
import os
import re
import json
from collections import defaultdict
from datetime import datetime, timezone
import pytz

In [ ]:
# the metrics calculated will be saved in '../metrics.csv'

In [3]:
repos_data_path = "/Users/zhangyujin/Desktop/repos-data"
metrics_path = "../metrics.csv"
metrics = pd.read_csv(metrics_path)

In [ ]:
# 01 repo_size

In [6]:
metrics['stars'] = None
metrics['forks'] = None
metrics['watchers'] = None
metrics['open_issues'] = None
metrics['network_count'] = None
metrics['subscribers_count'] = None
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    repo_path = f"{repos_data_path}/{owner}_{repo}"
    if os.path.exists(repo_path):
        repo_info_path = f"{repos_data_path}/{owner}_{repo}/repo.json"
        if os.path.exists(repo_info_path):
            with open(repo_info_path, 'r') as f:
                repo_info = json.load(f)
            metrics.at[index, 'stars'] = f"{repo_info.get('stargazers_count', 0):d}"
            metrics.at[index, 'forks'] = f"{repo_info.get('forks_count', 0):d}"
            metrics.at[index, 'watchers'] = f"{repo_info.get('watchers', 0):d}"
            metrics.at[index, 'open_issues'] = f"{repo_info.get('open_issues', 0):d}"
            metrics.at[index, 'network_count'] = f"{repo_info.get('network_count', 0):d}"
            metrics.at[index, 'subscribers_count'] = f"{repo_info.get('subscribers_count', 0):d}"

In [55]:
metrics['created_time'] = None
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    repo_path = f"{repos_data_path}/{owner}_{repo}"
    if os.path.exists(repo_path):
        repo_info_path = f"{repos_data_path}/{owner}_{repo}/repo.json"
        if os.path.exists(repo_info_path):
            with open(repo_info_path, 'r') as f:
                repo_info = json.load(f)
                created_at = datetime.strptime(repo_info['created_at'], '%Y-%m-%dT%H:%M:%SZ')
                today = datetime.strptime("2024-03-31T23:59:59Z", '%Y-%m-%dT%H:%M:%SZ')
            metrics.at[index, 'created_time'] = f"{(today-created_at).total_seconds()/(3600*24):.2f}"

In [79]:
metrics.to_csv(metrics_path, index=False)

In [ ]:
# 02 longitudinal(Recent 500 issues/pulls/commits)

In [51]:
metrics['issue_closed_rate'] = None
metrics['issue_time_to_close'] = None
metrics['issue_time_to_response'] = None
metrics['issue_monthly'] = None
metrics['issue_participants'] = None
metrics['issue_per_participants']= None

In [73]:
metrics['issue_commented_rate'] = None

In [4]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        closed_issues_count = 0
        labeled_issues_count = 0
        commented_issues_count = 0
        total_comments = 0
        total_time_to_comment = 0
        total_time_to_close = 0
        monthly_issues_count = defaultdict(int)
        participants = set()
        issues_count = 0
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
            
        for issue in issues[:500]:
            issues_count += 1
#             labeled_issues_count += 1 if issue['labels'] else 0
#             created_at = datetime.datetime.strptime(issue['created_at'], '%Y-%m-%dT%H:%M:%SZ')
#             monthly_issues_count[(created_at.year, created_at.month)] += 1

            if issue['state'] == 'closed':
                closed_issues_count += 1
#                 closed_at = datetime.datetime.strptime(issue['closed_at'], '%Y-%m-%dT%H:%M:%SZ')
#                 total_time_to_close += (closed_at - created_at).total_seconds()

#             comments_path = f"{repos_data_path}/{owner}_{repo}/comments/{issue['number']}.json"
            
#             if os.path.exists(comments_path):
                
#                 with open(comments_path, 'r') as f:
#                     comments = json.load(f)
                    
#                 total_comments += len(comments)
#                 first_comment_time = datetime.datetime.strptime(comments[0]['created_at'], '%Y-%m-%dT%H:%M:%SZ')
#                 time_to_comment = (first_comment_time - created_at).total_seconds()
#                 total_time_to_comment += time_to_comment
#                 commented_issues_count += 1
                
#                 for comment in comments:
#                     participant = comment['user']['login']
#                     participants.add(participant)
        
    metrics.at[index, 'issue_closed_rate'] = f"{(closed_issues_count / issues_count):.2f}" if issues_count > 0 else "N/A"
#     metrics.at[index, 'issue_time_to_close'] = f"{(total_time_to_close / closed_issues_count / (24 * 3600)):.1f}" if closed_issues_count > 0 else "N/A"
#     metrics.at[index, 'issue_time_to_response'] = f"{(total_time_to_comment / commented_issues_count / (24 * 3600)):.1f}" if commented_issues_count > 0 else "N/A"
#     metrics.at[index, 'issue_monthly'] = f"{(sum(monthly_issues_count.values()) / len(monthly_issues_count)):.1f}" if len(monthly_issues_count) > 0 else "N/A"
#     metrics.at[index, 'issue_participants'] = f"{len(participants):d}"
#     metrics.at[index, 'issue_per_participants'] = f"{(len(participants) / issues_count):.1f}" if issues_count > 0 else "N/A"
#     metrics.at[index, 'issue_commented_rate'] = f"{(commented_issues_count / issues_count):.2f}" if issues_count > 0 else "N/A"
    

In [10]:
metrics['labeled_issues'] = None

In [12]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        labeled_issues_count = 0
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
            
        for issue in issues[:500]:
            labeled_issues_count += 1 if issue['labels'] else 0
        
    metrics.at[index, 'labeled_issues'] = f"{labeled_issues_count:d}"

In [53]:
metrics['pulls_closed_rate'] = None
metrics['pulls_merged_rate'] = None
metrics['pulls_time_to_close'] = None
metrics['pulls_time_to_response'] = None
metrics['pulls_monthly'] = None
metrics['pulls_participants'] = None
metrics['pulls_per_participants'] = None

In [54]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    pulls_path = f"{repos_data_path}/{owner}_{repo}/pulls.json"
    
    if os.path.exists(pulls_path):
        pulls_count = 0
        closed_pulls_count = 0
        merged_pulls_count = 0
        commented_pulls_count = 0
        total_comments = 0
        total_time_to_comment = 0
        total_time_to_close = 0
        monthly_pulls_count = defaultdict(int)
        participants = set()
        
        with open(pulls_path, 'r') as f:
            pulls = json.load(f)
            
        if pulls is None:
            continue
            
        for pull in pulls[:500]:
            
            pulls_count += 1
            
            if pull['merged_at']:
                merged_pulls_count += 1
                
            created_at = datetime.datetime.strptime(pull['created_at'], '%Y-%m-%dT%H:%M:%SZ')
            monthly_pulls_count[(created_at.year, created_at.month)] += 1

            if pull['state'] == 'closed':
                closed_pulls_count += 1
                closed_at = datetime.datetime.strptime(pull['closed_at'], '%Y-%m-%dT%H:%M:%SZ')
                total_time_to_close += (closed_at - created_at).total_seconds()

            comments_path = f"{repos_data_path}/{owner}_{repo}/comments/{pull['number']}.json"
            
            if os.path.exists(comments_path):
                
                with open(comments_path, 'r') as f:
                    comments = json.load(f)
                    
                total_comments += len(comments)
                first_comment_time = datetime.datetime.strptime(comments[0]['created_at'], '%Y-%m-%dT%H:%M:%SZ')
                time_to_comment = (first_comment_time - created_at).total_seconds()
                total_time_to_comment += time_to_comment
                commented_pulls_count += 1
                
                for comment in comments:
                    participant = comment['user']['login']
                    participants.add(participant)
        
    metrics.at[index, 'pulls_closed_rate'] = f"{(closed_pulls_count / pulls_count):.1f}" if pulls_count > 0 else "N/A"  
    metrics.at[index, 'pulls_merged_rate'] = f"{(merged_pulls_count / closed_pulls_count):.1f}" if closed_pulls_count > 0 else "N/A"
    metrics.at[index, 'pulls_time_to_close'] = f"{(total_time_to_close / closed_pulls_count / (24 * 3600)):.1f}" if closed_pulls_count > 0 else "N/A"
    metrics.at[index, 'pulls_time_to_response'] = f"{(total_time_to_comment / commented_pulls_count / (24 * 3600)):.1f}" if commented_pulls_count > 0 else "N/A"
    metrics.at[index, 'pulls_monthly'] = f"{(sum(monthly_pulls_count.values()) / len(monthly_pulls_count)):.1f}" if len(monthly_pulls_count) > 0 else "N/A"
    metrics.at[index, 'pulls_participants'] = f"{len(participants):d}"
    metrics.at[index, 'pulls_per_participants'] = f"{(len(participants) / pulls_count):.1f}" if pulls_count > 0 else "N/A"  

In [18]:
metrics['issues_average_comments'] = None
metrics['pulls_average_comments'] = None

In [7]:
metrics['issue_most_recent_time'] = None

In [10]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        total_comments = 0
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
        metrics.at[index, 'issue_most_recent_time'] = f"{issues[0]['created_at']}" if issues else "N/A"

In [21]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    pulls_path = f"{repos_data_path}/{owner}_{repo}/pulls2.json"
    
    if os.path.exists(pulls_path):
        total_comments = 0
        
        with open(pulls_path, 'r') as f:
            pulls = json.load(f)
            
        for pull in pulls[:500]:
            total_comments += pull['comments']
        
    metrics.at[index, 'pulls_average_comments'] = f"{(total_comments / len(pulls)):.1f}" if pulls else "N/A"

In [42]:
metrics['code_review_score'] = None
metrics['maintained_score'] = None

In [43]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    scorecard_path = f"{repos_data_path}/{owner}_{repo}/scorecard.json"
    if os.path.exists(scorecard_path):
        with open(scorecard_path, 'r') as f:
            scorecard = json.load(f)
    if scorecard and scorecard['checks']:
        for check in scorecard['checks']:
            if check['name'] == "Code-Review":
                metrics.at[index, 'code_review_score'] = f"{check['score']}"
            if check['name'] == "Maintained":
                metrics.at[index, 'maintained_score'] = f"{check['score']}"

In [48]:
metrics['vulnerabilities']= None

In [49]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    scorecard_path = f"{repos_data_path}/{owner}_{repo}/scorecard.json"
    if os.path.exists(scorecard_path):
        with open(scorecard_path, 'r') as f:
            scorecard = json.load(f)
    if scorecard and scorecard['checks']:
        for check in scorecard['checks']:
            if check['name'] == "Vulnerabilities":
                metrics.at[index, 'vulnerabilities'] = f"{check['reason'].split()[0]}"

In [57]:
metrics['dependent_repositories'] = None
metrics['dependent_packages'] = None

In [58]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    dependents_path = f"{repos_data_path}/{owner}_{repo}/dependents.json"
    if os.path.exists(dependents_path):
        with open(dependents_path, 'r') as f:
            dependents = json.load(f)
        metrics.at[index, 'dependent_repositories'] = f"{dependents.get('repositories', '0')}"
        metrics.at[index, 'dependent_packages'] = f"{dependents.get('packages', 0)}"

In [70]:
metrics['commits_with_vertification_rate'] = None
metrics['commits_avg_msg_length'] = None
metrics['commits_author_count'] = None
metrics['commits_committer_count'] = None

In [71]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    commits_path = f"{repos_data_path}/{owner}_{repo}/commits.json"
    if os.path.exists(commits_path):
        commits_with_vertification_count = 0
        commits_count = 0
        commits_msg_length = 0
        authors = set()
        committers = set()
        with open(commits_path, 'r') as f:
            commits = json.load(f)
        if commits is None:
            continue
        for commit in commits:
            commits_count += 1
            if commit['author']:
                authors.add(commit['author']['login'])
            if commit['committer']:
                committers.add(commit['committer']['login'])
            commits_msg_length += len(commit['commit']['message'])
            if commit['commit']['verification']['verified'] is True:
                commits_with_vertification_count += 1
        metrics.at[index, 'commits_with_vertification_rate'] = f"{(commits_with_vertification_count / commits_count):.1f}" if commits_count > 0 else "N/A"  
        metrics.at[index, 'commits_avg_msg_length'] = f"{(commits_msg_length / commits_count):.1f}" if commits_count > 0 else "N/A"  
        metrics.at[index, 'commits_author_count'] = f"{len(authors):d}"
        metrics.at[index, 'commits_committer_count'] = f"{len(committers):d}"

In [76]:
metrics['readme_headers_num'] = None

In [80]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    readme_path = f"{repos_data_path}/{owner}_{repo}/readme.md"
    if os.path.exists(readme_path):
        heading_levels = {
            'h1': re.compile(r'^(#\s.*)', re.MULTILINE),
            'h2': re.compile(r'^(##\s.*)', re.MULTILINE),
            'h3': re.compile(r'^(###\s.*)', re.MULTILINE)
        }
        counts = 0
        with open(readme_path, 'r', encoding='utf-8') as file:
            for line in file:
                for level, regex in heading_levels.items():
                    matches = regex.findall(line)
                    counts += len(matches)
        metrics.at[index, 'readme_headers_num'] = counts 

In [3]:
metrics['comments_avg_length'] = None

In [4]:
for index, row in metrics.iterrows():
    
    owner = row['owner']
    repo = row['repo']
    
    issues_path = f"{repos_data_path}/{owner}_{repo}/issues.json"
    
    if os.path.exists(issues_path):
        comments_count = 0
        comments_length = 0
        
        with open(issues_path, 'r') as f:
            issues = json.load(f)
            
        for issue in issues[:500]:
            comments_path = f"{repos_data_path}/{owner}_{repo}/comments/{issue['number']}.json"
            
            if os.path.exists(comments_path):
                with open(comments_path, 'r') as f:
                    comments = json.load(f)
                for comment in comments:
                    comments_length += len(comment['body'])
                    comments_count += 1
        
    metrics.at[index, 'comments_avg_length'] = f"{(comments_length / comments_count):.2f}" if comments_count > 0 else "N/A"

In [6]:
metrics['pulls_last_90days'] = None

In [16]:
for index, row in metrics.iterrows():
    target_date_aware = datetime(2023, 12, 1, tzinfo=pytz.utc)
    owner = row['owner']
    repo = row['repo']
    pulls_count = 0
    pulls_path = f"{repos_data_path}/{owner}_{repo}/pulls.json"
    if os.path.exists(pulls_path):
        with open(pulls_path, 'r') as f:
            pulls = json.load(f)
        if pulls is None:
            continue
        for pull in pulls[:500]:
            date_str = pull['created_at']
            date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            date_obj_aware = date_obj.replace(tzinfo=pytz.utc)
            if date_obj_aware > target_date_aware:
                pulls_count += 1
    metrics.at[index, 'pulls_last_90days'] = f"{pulls_count:d}" if pulls_count > 0 else "N/A"
    

In [19]:
metrics['commmits_last_90days'] = None

In [20]:
for index, row in metrics.iterrows():
    target_date_aware = datetime(2023, 12, 1, tzinfo=pytz.utc)
    owner = row['owner']
    repo = row['repo']
    commits_count = 0
    commits_path = f"{repos_data_path}/{owner}_{repo}/commits.json"
    if os.path.exists(commits_path):
        with open(commits_path, 'r') as f:
            commits = json.load(f)
        if commits is None:
            continue
        for commit in commits[:500]:
            date_str = commit['commit']['committer']['date']
            date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            date_obj_aware = date_obj.replace(tzinfo=pytz.utc)
            if date_obj_aware > target_date_aware:
                commits_count += 1
    metrics.at[index, 'commmits_last_90days'] = f"{commits_count:d}" if commits_count > 0 else "N/A"
    

In [4]:
metrics['commit_most_recent_time'] = None

In [5]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    commits_path = f"{repos_data_path}/{owner}_{repo}/commits.json"
    if os.path.exists(commits_path):
        with open(commits_path, 'r') as f:
            commits = json.load(f)
        metrics.at[index, 'commit_most_recent_time'] = f"{commits[0]['commit']['author']['date']}" if commits else "N/A"

In [7]:
metrics['health_percentage'] = None

In [10]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    profile_path = f"{repos_data_path}/{owner}_{repo}/profile.json"
    if os.path.exists(profile_path):
        with open(profile_path, 'r') as f:
            profile = json.load(f)
        metrics.at[index, 'health_percentage'] = f"{profile.get('health_percentage')}" if profile else "N/A"

In [20]:
metrics['projects_owned_by_maintainer'] = None
metrics['age_of_other_projects'] = None

In [21]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    maintainer_path = f"{repos_data_path}/{owner}_{repo}/maintainer.json"
    today = datetime.strptime("2024-03-31T23:59:59Z", '%Y-%m-%dT%H:%M:%SZ')
    age = 0
    if os.path.exists(maintainer_path):
        with open(maintainer_path, 'r') as f:
            maintainer = json.load(f)
        for repo in maintainer:
            if isinstance(repo, dict):
                created_at = datetime.strptime(repo['created_at'], '%Y-%m-%dT%H:%M:%SZ')
                age += (today - created_at).total_seconds()
    metrics.at[index, 'projects_owned_by_maintainer'] = f"{len(maintainer)}" if maintainer else "N/A"
    metrics.at[index, 'age_of_other_projects'] = f"{age / len(maintainer) / (3600*24):.1f}" if maintainer else "N/A"

In [23]:
metrics['workflow_runs_count'] = None

In [24]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    workflow_path = f"{repos_data_path}/{owner}_{repo}/workflow.json"
    if os.path.exists(workflow_path):
        with open(workflow_path, 'r') as f:
            workflow = json.load(f)
    metrics.at[index, 'workflow_runs_count'] = f"{workflow['total_count']}" if workflow else "N/A"

In [27]:
metrics['contributors_per_code_file'] = None
metrics['files_with_more_than_1_contributors'] = None

In [67]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    commits_path = f"{repos_data_path}/{owner}_{repo}/commits.json"
    if os.path.exists(commits_path):
        with open(commits_path, 'r') as f:
            commits = json.load(f)
        if commits is None:
            continue
        file_contributors = {}
        for commit in commits:
            commit_sha = commit['sha']
            commit_path = f"{repos_data_path}/{owner}_{repo}/commits/{commit_sha}.json"
            if os.path.exists(commit_path):
                with open(commit_path, 'r') as f:
                    commit_data = json.load(f)
                if commit_data.get('author'):
                    author = commit_data['author']['login']
                    for file in commit_data['files']:
                        filename = file['filename']
                        if filename not in file_contributors:
                            file_contributors[filename] = set()
                        file_contributors[filename].add(author)
        total_contributors = 0
        total_files = 0
        files_with_more_than_1_contributors = 0
        for contributors in file_contributors.values():
            total_contributors += len(contributors)
            total_files += 1
            if len(contributors) > 1:
                files_with_more_than_1_contributors += 1
        num_files = len(file_contributors)
        metrics.at[index, 'contributors_per_code_file'] = total_contributors / total_files if total_files > 0 else "N/A"
#         metrics.at[index, 'files_with_more_than_1_contributors'] = f"{files_with_more_than_1_contributors}" if files_with_more_than_1_contributors > 0 else "N/A" 
        

In [34]:
metrics['labels'] = None

In [35]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    labels_path = f"{repos_data_path}/{owner}_{repo}/labels.json"
    if os.path.exists(labels_path):
        with open(labels_path, 'r') as f:
            labels = json.load(f)
    metrics.at[index, 'labels'] = f"{len(labels)}" if labels else "N/A"

In [37]:
metrics['releases'] = None

In [38]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    releases_path = f"{repos_data_path}/{owner}_{repo}/releases.json"
    if os.path.exists(releases_path):
        with open(releases_path, 'r') as f:
            releases = json.load(f)
    metrics.at[index, 'releases'] = f"{len(releases)}" if releases else "N/A"

In [57]:
metrics['pull_authors_dec'] = None
metrics['pull_authors_jan'] = None
metrics['pull_difference'] = None

In [60]:
for index, row in metrics.iterrows():
    dec_aware = datetime(2023, 12, 1, tzinfo=pytz.utc)
    jan_aware = datetime(2024, 1, 1, tzinfo=pytz.utc)
    feb_aware = datetime(2024, 2, 1, tzinfo=pytz.utc)
    owner = row['owner']
    repo = row['repo']
    commits_count = 0
    pulls_path = f"{repos_data_path}/{owner}_{repo}/pulls.json"
    if os.path.exists(pulls_path):
        with open(pulls_path, 'r') as f:
            pulls = json.load(f)
        if pulls is None:
            continue
        dec_authors = set()
        jan_authors = set()
        for pull in pulls[:500]:
            author = pull['user']['login']
            date_str = pull['created_at']
            date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            date_obj_aware = date_obj.replace(tzinfo=pytz.utc)
            if date_obj_aware > dec_aware and date_obj_aware < jan_aware:
                dec_authors.add(author)
            elif date_obj_aware > jan_aware and date_obj_aware < feb_aware:
                jan_authors.add(author)
    metrics.at[index, 'pull_authors_dec'] = f"{len(dec_authors):d}" if len(dec_authors) > 0 else "N/A"
    metrics.at[index, 'pull_authors_jan'] = f"{len(jan_authors):d}" if len(jan_authors) > 0 else "N/A"
    metrics.at[index, 'pull_difference'] = f"{len(jan_authors)-len(dec_authors):d}"

In [62]:
metrics['pull_authors_2024'] = None

In [63]:
for index, row in metrics.iterrows():
    jan_aware = datetime(2024, 1, 1, tzinfo=pytz.utc)
    owner = row['owner']
    repo = row['repo']
    commits_count = 0
    pulls_path = f"{repos_data_path}/{owner}_{repo}/pulls.json"
    if os.path.exists(pulls_path):
        with open(pulls_path, 'r') as f:
            pulls = json.load(f)
        if pulls is None:
            continue
        authors = set()
        for pull in pulls[:500]:
            author = pull['user']['login']
            date_str = pull['created_at']
            date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
            date_obj_aware = date_obj.replace(tzinfo=pytz.utc)
            if date_obj_aware > jan_aware:
                authors.add(author)
    metrics.at[index, 'pull_authors_2024'] = f"{len(authors):d}" if len(authors) > 0 else "N/A"

In [69]:
metrics['Usage_Popularity'] = None
for index, row in metrics.iterrows():
    metrics.at[index, 'Usage_Popularity'] = (metrics.at[index, 'stars']+metrics.at[index, 'watchers']+metrics.at[index, 'forks']+metrics.at[index, 'dependent_repositories'])/4 


In [78]:
metrics['headings_count'] = None

In [ ]:
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    contributing_path = f"{repos_data_path}/{owner}_{repo}/CONTRIBUTING.md"
    code_of_conduct_path = f"{repos_data_path}/{owner}_{repo}/code-of-conduct.md"
    governance_path = f"{repos_data_path}/{owner}_{repo}/GOVERNANCE.md"
    counts = 0
    if os.path.exists(contributing_path):
        heading_levels = {
            'h1': re.compile(r'^(#\s.*)', re.MULTILINE),
            'h2': re.compile(r'^(##\s.*)', re.MULTILINE),
            'h3': re.compile(r'^(###\s.*)', re.MULTILINE)
        }
        with open(contributing_path, 'r', encoding='utf-8') as file:
            for line in file:
                for level, regex in heading_levels.items():
                    matches = regex.findall(line)
                    counts += len(matches)
    if os.path.exists(code_of_conduct_path):
        heading_levels = {
            'h1': re.compile(r'^(#\s.*)', re.MULTILINE),
            'h2': re.compile(r'^(##\s.*)', re.MULTILINE),
            'h3': re.compile(r'^(###\s.*)', re.MULTILINE)
        }
        with open(code_of_conduct_path, 'r', encoding='utf-8') as file:
            for line in file:
                for level, regex in heading_levels.items():
                    matches = regex.findall(line)
                    counts += len(matches)
    if os.path.exists(governance_path):
        heading_levels = {
            'h1': re.compile(r'^(#\s.*)', re.MULTILINE),
            'h2': re.compile(r'^(##\s.*)', re.MULTILINE),
            'h3': re.compile(r'^(###\s.*)', re.MULTILINE)
        }
        with open(governance_path, 'r', encoding='utf-8') as file:
            for line in file:
                for level, regex in heading_levels.items():
                    matches = regex.findall(line)
                    counts += len(matches)
        metrics.at[index, 'readme_headers_num'] = counts 